This notebook is designed to walk the user through the classes included in lvlspy. This first thing to do is to quietly install and import any missing python packages this notebook will use

In [1]:
import sys, subprocess, pkg_resources
required = {'lvlspy'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed
if missing:
    subprocess.check_call([sys.executable, '-m','pip','install','--quiet',*missing])

import lvlspy.level as lv
import lvlspy.spcoll as lc
import lvlspy.species as ls
import lvlspy.transition as lt


You can create an array of levels with input consisting of energy and multiplicity. The unit is optional and defaults to keV if unspecified. If specified, the API will convert to keV for internal use.

In [2]:
levs = [lv.Level(0,1),lv.Level(1,1,units="GeV"),lv.Level(500,3,units="eV"),lv.Level(200,4,units = "MeV"),lv.Level(5,2)]

You can associate the levels with a particular species. For our purposes, we shall call it 'test' and print out the levels. The species module with automatically order the energies in ascending order. The printout is in units of keV as mentioned.

In [3]:
s = ls.Species('test',levels=levs)
for lev in s.get_levels():
    print(lev.get_energy(), lev.get_multiplicity())

0.0 1
0.5 3
5.0 2
200000.0 4
1000000.0 1


Levels can be added or removed at any point via the following commands. Care should be given if the unit has been specified beforehand. If you included a unit while listing the levels, you must include it while removing, since it will throw an error stating it couldn't find the level you wish to remove. 

In [4]:
# Add another level.

s.add_level(lv.Level(300, 3))

# Remove a level
s.remove_level(lv.Level(200, 4,units = 'MeV')) 

# Print out levels again. 

for lev in s.get_levels():
    print(lev.get_energy(), lev.get_multiplicity())


0.0 1
0.5 3
5.0 2
300.0 3
1000000.0 1


You can create and update optional properties, physical or custom, for any level. In this example, we will update properties for level 0.  Updating the property will create it if it does not already exist or change to the new value if it already does (see the *smell* property).

In [5]:
levs[0].update_properties({'color': 'black'})
levs[0].update_properties({'smell': 'unpleasant'})
levs[0].update_properties({'smell': 'nice'})
levs[0].update_properties({'first name': 'Jaad'})
levs[0].update_properties({'last name': 'Tannous'})

# Print out level properties

for prop in levs[0].get_properties():
    print(prop, ':', levs[0].get_properties()[prop])

color : black
smell : nice
first name : Jaad
last name : Tannous


The same can be applied to species.

In [6]:
s.update_properties({'Motto': 'Go Tigers!'})
s.update_properties({('key1', 'key2', 'key3'): 'value'})

# Print out species properties

for prop in s.get_properties():
    print(prop, ':', s.get_properties()[prop])

Motto : Go Tigers!
('key1', 'key2', 'key3') : value


You can calculate transition properties between any two energy states. Since we can experimentally attain Einstein A coefficients, the method takes downward transitioning states with a supplied Einstein A coefficient between said states. Moreover, you can also append optional properties to said transition

In [7]:
t = lt.Transition(levs[1], levs[0], 100.)
# Update optional properties for transition

t.update_properties({'Name': 'Fast'})

# Print out transition properties

for prop in t.get_properties():
    print(prop, ':', t.get_properties()[prop])


Name : Fast


The properties of the transition can be extracted. Properties include the energy of the states, the supplied Einstein A coefficient, and the calculated B coefficients.

In [8]:
# Print the Einstein coefficients for the transition

print(t.get_upper_level().get_energy(), t.get_lower_level().get_energy(),
      t.get_Einstein_A(), t.get_Einstein_B_upper_to_lower(),
      t.get_Einstein_B_lower_to_upper())

# Print the frequency for the transition

print('nu (per second) =', t.get_frequency())

1000000.0 0.0 100.0 4.797249137368367e-22 4.797249137368367e-22
nu (per second) = 2.4179892822261806e+23


For a given temperature, in Kelvin, you can compute the upward and downward transition rates between the two states. You can also calculate the probabilities of finding the species at a given level with specified temperature, when equilibrium is achieved.

In [9]:
T = 1.e9

print(t.compute_upper_to_lower_rate(T), t.compute_lower_to_upper_rate(T))

# Compute and print out the equilibrium probabilities

p = s.compute_equilibrium_probabilities(T)

for i in range(len(p)):
    print('Level =', i, ', Probability =', p[i])

100.0 0.0
Level = 0 , Probability = 0.16772338183111088
Level = 1 , Probability = 0.500259075515663
Level = 2 , Probability = 0.316537170049882
Level = 3 , Probability = 0.015480372603344157
Level = 4 , Probability = 0.0


/Users/bradleymeyer/Desktop/work/lvlspy/lvlspy/transition.py:158: RuntimeWarning: overflow encountered in exp
  return self._fnu() / (np.exp((deltaE * u.keV / T_keV).value) - 1.0)


You can also add the transitions to the species 

In [10]:
s.add_transition(t)

print('New number of transitions =', len(s.get_transitions()))

for tr in s.get_transitions():
    print('A:', t.get_Einstein_A())


New number of transitions = 1
A: 100.0


The rate matrix can be calculated for the number of transitions and temperature supplied

In [11]:
rate_matrix = s.compute_rate_matrix(T)

print('\nRate Matrix:\n')

for i in range(rate_matrix.shape[0]):
    for j in range(rate_matrix.shape[1]):
        print(i, j, rate_matrix[i, j])


Rate Matrix:

0 0 0.0
0 1 0.0
0 2 0.0
0 3 0.0
0 4 100.0
1 0 0.0
1 1 0.0
1 2 0.0
1 3 0.0
1 4 0.0
2 0 0.0
2 1 0.0
2 2 0.0
2 3 0.0
2 4 0.0
3 0 0.0
3 1 0.0
3 2 0.0
3 3 0.0
3 4 0.0
4 0 0.0
4 1 0.0
4 2 0.0
4 3 0.0
4 4 -100.0


The API allows to print out the species collection, and all their properties, into xml format. The output files in turn can be used with separate scripts to do various calculations. To illustrate writing to XML, let's create another species and species collection

In [12]:
#New set of levels
levs = [lv.Level(0,1), lv.Level(100,1),lv.Level(500,3),lv.Level(200,4)]

#New species called 'test2'
s2 = ls.Species('test2',levels = levs)

#Create collection
my_coll = lc.SpColl([s,s2])

#print them out
for ss in my_coll.get():
    print(ss)

test
test2


You can also add optional properties to the collection

In [13]:
my_coll.update_properties({'color' : 'red'})

print(my_coll.get_properties())

{'color': 'red'}


You can write your collection to XML

In [14]:
my_coll.write_to_xml('test.xml')

!cat test.xml

<species_collection>
  <optional_properties>
    <property name="color">red</property>
  </optional_properties>
  <species name="test">
    <optional_properties>
      <property name="Motto">Go Tigers!</property>
      <property name="key1" tag1="key2" tag2="key3">value</property>
    </optional_properties>
    <levels>
      <level>
        <optional_properties>
          <property name="color">black</property>
          <property name="smell">nice</property>
          <property name="first name">Jaad</property>
          <property name="last name">Tannous</property>
        </optional_properties>
        <properties>
          <energy>0.0</energy>
          <multiplicity>1</multiplicity>
        </properties>
      </level>
      <level>
        <properties>
          <energy>0.5</energy>
          <multiplicity>3</multiplicity>
        </properties>
      </level>
      <level>
        <properties>
          <energy>5.0</energy>
          <multiplicity>2</multiplicity>
        </pro